# EC2 T 타입 정리
- Burstable performance instances
    - 다른 인스턴스 타입처럼 고정된 용량의 CPU 자원을 제공하지 않음
    - CPU 크레딧 제도
        - CPU 사용량이 기본 수준(Baseline) 이상(burst) 이면 크레딧 차감
        - CPU 사용량이 기본 수준(Baseline) 미만이라면 크레딧을 제공
    - Baseline : CPU 사용량을 기준, 크레딧의 소모와 증가가 같은 지점
        - 인스턴스의 크기에 따라 다름
- 즉 평소에는 baseline 밑으로 유지해 크레딧을 모으고 꼭 필요한 상황에 크레딧을 소모해 Burst
- 크레딧이 없다면 최악의 경우 CPU 사용량이 5% 미만으로 제한됨

## CPU 크레딧과 베이스라인
- CPU 사용이 베이스라인보다 적은 경우 : 크레딧 증가
- CPU 사용이 베이스라인보다 많은 경우 : 크레딧 소모
- CPU 사용이 베이스라인과 같은 경우 : 크레딧 증감 없음
- CPU 크레딧 : vCPU가 사용하는 시간 단위
    - 1 CPU credit : 1 vCPU * 100% utilization * 1 minute
    - 1 CPU credit : 1 vCPU * 50% utilization * 2 minutes
    - 1 CPU credit : 2 vCPU * 25& utilization * 2 minutes
- 시간당 크레딧의 추가 : 각 인스턴스 별로 지정된 베이스라인을 기준으로 공식 산출
    - 공식 : vCPU 개수 * 베이스라인 * 60분
    - 예 : T3.nano(baseline:5%) 경우 시간당 6크레딧 부여
        - 2 vCPU * 5% * 60 minute : 6 credits

## CPU 크레딧의 추가와 소모
- CPU 크레딧은 CPU 사용량이 베이스라인 미만일 때 지속적으로 추가 (millisecond 단위)
- CPU 사용량이 베이스라인 이상일 때 지속적으로 소모 (millisecond 단위)
- 각 인스턴스 사이즈 별로 크레딧 최대 저장 한계가 존재
    - 일반적으로 시간당 얻을 수 있는 최대 크레딧 * 24
    - 예 : t3.nano의 경우 6(시간당 얻는 크레딧) * 24 = 144 크레딧
    - 이 이상 얻는 크레딧은 버려짐

## T 인스턴스 모드
- 스탠다드 모드
    - 베이스라인 이상으로 버스트가 발생할 경우, 미리 저장된 크레딧을 소모해 CPU 사용
    - 크레딧이 없다면 베이스라인 이상으로 CPU 사용 불가
- 언리미티드 모드
    - 제한없이 필요한만큼 CPU 사용
    - 베이스라인 이상으로 버스트가 발생할 경우, 미리 저장된 크레딧을 소모해 사용
    - 크레딧이 없을 경우, 24시간 안으로 크레딧을 빌려 충당
    - 24시간 안에 베이스라인 미만으로 CPU를 유지해 추가된 크레딧으로 갚기 가능
    - 갚지 못한 크레딧은 비용을 지불해 충당
- T2는 스탠다드가 기본, T3는 언리미티드가 기본

## Launch Credit
- T2 인스턴스의 경우 인스턴스가 처음 생성될 때 Launch Credit 제공
    - 생성 직후에 버스트로 진입할 경우를 대비
    - 스탠다드 모드가 디폴트이기 때문
- T3의 경우 언미리미티드 모드가 디폴트이기 때문에 Launch Credit이 제공되지 않음
    - 즉 T3를 스탠다드 모드로 사용하면 생성 직후 버스트 진입 불가

## T2 인스턴스의 크레딧이 없는 경우
- 인스턴스의 응답이 없음
- 어플리케이션이 자주 멈춤
    - 웹서버의 경우 500 에러 등
    - 정상 동작하다가(크레딧 충분) 멈추다(크레딧 소진) 반복
- 해결책
    - 인스턴스 사이즈 늘리기
    - 인스턴스 타입 바꾸지
    - 언리미티드 모드로 바꾸기
    - 계획적으로 CPU 사용하기

## 결론
- T 인스턴스 (특히 T2) 타입의 경우에는 사용 계획이 필요함
    - CPU 사용량에 대해 잘 계획해서 사용하면 이득을 볼 수 있지만, 막 사용하면 오히려 손해를 볼 수 있음
    - 개발용 인스턴스 같은 경우에 T타입이 적절
    - 라이브 서비스를 위한 인스턴스의 경우 T타입을 사용할 때 주의가 필요
    - CPU Credit은 CloudWatch 지표로 모니터링 가능(알람 처리 가능)
- 사용량에 맞는 사이즈를 적절하게 고르자!
